In [1]:
from fastembed import SparseTextEmbedding
from RAG.RAG_query import *
import os

from RAG.RAG_query import get_topics_list
from config.loadConfig import load_project_config
from DB.qDrant import check_qdrant_status, QdrantConfig
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from RAG.RAG_Retrieve import custom_embed_function

# Encode the query text using the same model

# Connect to Qdrant
config = load_project_config(os.path.join(os.getcwd(), "config/config.yaml"))
qdrantConfig = QdrantConfig(
    host=config["db"]["qdrant"]["connection"]["host"],
    port=config["db"]["qdrant"]["connection"]["port"],
    timeout=config["db"]["qdrant"]["connection"]["timeout"]
)
status, client = check_qdrant_status(qdrantConfig)
embedding_model = SentenceTransformer(config['embedding_model'])
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm25")
small_collection_name = config['db']['collections']['collection_small']['name']
large_collection_name = config['db']['collections']['collection_large']['name']

Connected. Collections: ['wiki_large_chunks', 'wiki_small_chunks']


In [2]:
get_topics_list(client, num_topics=100)

008b7b75-2108-429e-8660-0ea16a0df3aa Academy Awards
00a690a3-2601-4f17-ac68-2e25dbfc830b Alain Connes
00a9e4e0-b1e6-474d-909a-ff2dc080ac7b Appellate procedure in the United States
00b0f22c-f37b-4670-b045-3018e271356f Austin (disambiguation)
00cd3a49-fdee-44c8-8925-9c25cf049ffd Anthropology
00dacad5-13f6-40af-a7c5-0849a919912f Afroasiatic languages
00fb841f-fb4e-4d0f-87cc-732dfd363900 Algeria
011e91c4-fd34-4f64-ba66-5aeb83788382 Apollo
0158886f-837d-4cb9-a7f0-d0fbb3288dbd Algeria
02250f07-9063-466f-bd3f-2ff49d0423b7 Andorra
02811e75-04e9-41e9-813f-095a28ea77e1 Andre Agassi
0289050d-5031-4f8f-ad0e-b8b485912547 Algeria
02b6acb8-fc28-4d9f-b6e5-f0d0e6a2433f Altruism
02c18c00-6ecd-48fa-9784-ca4c200c0f9e Afroasiatic languages
02d41de5-acb8-4d93-b2ee-b517dbe06898 Abraham Lincoln
02f35640-6356-4a14-8874-631251b3c494 Alchemy
02f97844-d768-44f9-b6c2-633341fc2f75 Albedo
030446c4-14dd-40f0-a8b3-2d8735791b64 Algeria
032679a7-bb08-4b16-91df-c6f66c5517b5 Ayn Rand
03403545-a248-44b9-80b3-22edd57f8067 A

In [3]:
query_text = "Who was Charlemagne?"
results = sparse_query_search(client, query_text, print_results=True)


--- : Querying Small Chunks for: 'Who was Charlemagne?' ---
The Economist featured a weekly column entitled "Charlemagne", focusing generally on European affairs and, more usually and specifically, on the European Union and its politics.  Actor and singer Christopher Lee's symphonic metal concept album Charlemagne: By the Sword and the Cross and its heavy metal follow-up Charlemagne: The Omens of Death feature the events of Charlemagne's life.
of his father, Pepin. Desiderius sent his own ambassadors denying the pope's charges. The ambassadors met at Thionville, and Charlemagne upheld the pope's side. Charlemagne demanded what the pope had requested, but Desiderius swore never to comply. Charlemagne and his uncle Bernard crossed the Alps in 773 and chased the Lombards back to Pavia, which they then besieged. Charlemagne temporarily
In April 2014, on the occasion of the 1200th anniversary of Charlemagne's death, public art Mein Karl by Ottmar Hörl at Katschhof place was installed betwe

In [4]:
query_text = "Who was Charlemagne?"
results = dense_query_search(client, query_text, print_results=True)


--- : Querying Small Chunks for: 'Who was Charlemagne?' ---
384
him. As an administrator, Charlemagne stands out for his many reforms: monetary, governmental, military, cultural and ecclesiastical. He is the main protagonist of the "Carolingian Renaissance".
Visigoths and Carolingians: the legend of Charlemagne
Visigoths and Carolingians: the legend of Charlemagne
In July 2022, Charlemagne featured as a character in an episode of The Family Histories Podcast, and it references his role as an ancestor of all modern Europeans. He is portrayed here in later life, and is speaking Latin, which is translated by a device. He is returned to 9th Century Aquitaine by the end of the episode after a DNA sample has been extracted.  Notes  References  Citations
Jews in Charlemagne's realm


In [14]:
hierarchical_search(client, query_text, use_deep_embedding=True, print_results=True)


--- : Querying Small Chunks for: 'Who was Charlemagne?' ---
384
Administration  Organisation The Carolingian king exercised the bannum, the right to rule and command. Under the Franks, it was a royal prerogative but could be delegated. He had supreme jurisdiction in judicial matters, made legislation, led the army, and protected both the Church and the poor. His administration was an attempt to organise the kingdom, church and nobility around him. As an administrator, Charlemagne stands out for his many reforms: monetary, governmental, military, cultural and ecclesiastical. He is the main protagonist of the "Carolingian Renaissance".  Military Charlemagne's success rested primarily on novel siege technologies and excellent logistics rather than the long-claimed "cavalry revolution" led by Charles Martel in 730s. However, the stirrup, which made the "shock cavalry" lance charge possible, was not introduced to the Frankish kingdom until the late eighth century.  Horses were used extensi

[Record(id='90c224c1-60e5-42a8-9cd0-69faafa396cd', payload={'text': 'Administration\n\nOrganisation\nThe Carolingian king exercised the bannum, the right to rule and command. Under the Franks, it was a royal prerogative but could be delegated. He had supreme jurisdiction in judicial matters, made legislation, led the army, and protected both the Church and the poor. His administration was an attempt to organise the kingdom, church and nobility around him. As an administrator, Charlemagne stands out for his many reforms: monetary, governmental, military, cultural and ecclesiastical. He is the main protagonist of the "Carolingian Renaissance".\n\nMilitary\nCharlemagne\'s success rested primarily on novel siege technologies and excellent logistics rather than the long-claimed "cavalry revolution" led by Charles Martel in 730s. However, the stirrup, which made the "shock cavalry" lance charge possible, was not introduced to the Frankish kingdom until the late eighth century.\n\nHorses were

In [5]:
result = client.retrieve(
    collection_name="wiki_large_chunks",
    ids=["035951e1-74e1-450e-a6a8-5bf9440fbad1"],
    with_payload=True
)
print(result)

[]
